In [15]:

import pandas as pd # gestion de datos 
import glob #modulo que sirve para buiscar archivos.
import time  # modulo que proporciona varias funciones relacionadas con el tiempo
import duckdb
import os
import numpy as np

In [2]:
#creamos la conexión con la base de datos:  #connect () crea una base de datos local por defecto que se borra al cerrar python.
#Si tenemos una base de datos y queremos solo leer los datos podemos usar el parámetro  .connect("ddbb", read_only=True)
conn=duckdb.connect()

In [3]:
cur_time=time.time() #devuelve la hora actual 
#!df=pd.concat([pd.read_csv(f)for f in glob.glob('dataset/*.csv')],axis=0, ignore_index=True) Si tenemos varios data set, y queremos juntarlos todos 
df=pd.read_csv('dataset/Sales_Product_Combined.csv')

In [4]:
df.head()

,Order ID,Product,Quantity Ordered,Price,Order Date,Time,Purchase Address,City,Product Type
0,176558,USB-C Charging Cable,2,11.95,19-04-2019,8:46 AM,"917 1st St, Dallas, TX 75001",Dallas,Cable
1,176559,Bose SoundSport Headphones,1,99.99,07-04-2019,10:30 PM,"682 Chestnut St, Boston, MA 02215",Boston,Headphones
2,176560,Google Phone,1,600,12-04-2019,2:38 PM,"669 Spruce St, Los Angeles, CA 90001",Los Angeles,Phone
3,176560,Wired Headphones,1,11.99,12-04-2019,2:38 PM,"669 Spruce St, Los Angeles, CA 90001",Los Angeles,Headphones
4,176561,Wired Headphones,1,11.99,30-04-2019,9:27 AM,"333 8th St, Los Angeles, CA 90001",Los Angeles,Headphones


In [7]:
cur_time=time.time()
query=''' 
SELECT *
FROM 'dataset/*.csv'
LIMIT 10'''
df_1=conn.execute(query).df() #.df nos devolverá un pandas dataframe
display(df)

,Order ID,Product,Quantity Ordered,Price,Order Date,Time,Purchase Address,City,Product Type
0,176558,USB-C Charging Cable,2,11.95,19-04-2019,8:46 AM,"917 1st St, Dallas, TX 75001",Dallas,Cable
1,176559,Bose SoundSport Headphones,1,99.99,07-04-2019,10:30 PM,"682 Chestnut St, Boston, MA 02215",Boston,Headphones
2,176560,Google Phone,1,600,12-04-2019,2:38 PM,"669 Spruce St, Los Angeles, CA 90001",Los Angeles,Phone
3,176560,Wired Headphones,1,11.99,12-04-2019,2:38 PM,"669 Spruce St, Los Angeles, CA 90001",Los Angeles,Headphones
4,176561,Wired Headphones,1,11.99,30-04-2019,9:27 AM,"333 8th St, Los Angeles, CA 90001",Los Angeles,Headphones
...,...,...,...,...,...,...,...,...,...
185945,259353,AAA Batteries (4-pack),3,2.99,17-09-2019,8:56 PM,"840 Highland St, Los Angeles, CA 90001",Los Angeles,Batteries
185946,259354,iPhone,1,700,01-09-2019,4:00 PM,"216 Dogwood St, San Francisco, CA 94016",San Francisco,iPhone
185947,259355,iPhone,1,700,23-09-2019,7:39 AM,"220 12th St, San Francisco, CA 94016",San Francisco,iPhone
185948,259356,34in Ultrawide Monitor,1,379.99,19-09-2019,5:30 PM,"511 Forest St, San Francisco, CA 94016",San Francisco,Monitor


In [11]:
#Creamos una vista de la querie anterior:
conn.register("df_view", df_1) 
#Usamos el método DESCRIBE para obtener los tipos de datos de nuestras collumnas
conn.execute("DESCRIBE df_view").df()

,column_name,column_type,null,key,default,extra
0,Order ID,BIGINT,YES,None,None,None
1,Product,VARCHAR,YES,None,None,None
2,Quantity Ordered,BIGINT,YES,None,None,None
3,Price,VARCHAR,YES,None,None,None
4,Order Date,TIMESTAMP,YES,None,None,None
5,Time,VARCHAR,YES,None,None,None
6,Purchase Address,VARCHAR,YES,None,None,None
7,City,VARCHAR,YES,None,None,None
8,Product Type,VARCHAR,YES,None,None,None


In [14]:
# Para encontrar los valores nulos necesitamos todos los datos. Para ello creamos una view nueva con toda la seleccion.
query_1=""" 
SELECT *
FROM 'dataset/*.csv'"""
df_2=conn.execute(query_1).df()


In [16]:
#calculamos el porcentaje de nulos:
porc_nulos=lambda df:(df.isnull().sum()/df.shape[0])*100
porc_nulos(df_2)

Order ID            0.0
Product             0.0
Quantity Ordered    0.0
Price               0.0
Order Date          0.0
Time                0.0
Purchase Address    0.0
City                0.0
Product Type        0.0
dtype: float64

In [17]:
df_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 185950 entries, 0 to 185949
Data columns (total 9 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   Order ID          185950 non-null  int64         
 1   Product           185950 non-null  object        
 2   Quantity Ordered  185950 non-null  int64         
 3   Price             185950 non-null  object        
 4   Order Date        185950 non-null  datetime64[us]
 5   Time              185950 non-null  object        
 6   Purchase Address  185950 non-null  object        
 7   City              185950 non-null  object        
 8   Product Type      185950 non-null  object        
dtypes: datetime64[us](1), int64(2), object(6)
memory usage: 12.8+ MB


In [22]:
query_3= """
SELECT *
FROM df_2
WHERE "Order Id"= '295665' 
"""
conn.execute(query_3).df()

The history saving thread hit an unexpected error (OperationalError('database or disk is full')).History will not be written to the database.


,Order ID,Product,Quantity Ordered,Price,Order Date,Time,Purchase Address,City,Product Type
0,295665,Macbook Pro Laptop,1,"1,700.00",2019-12-30,12:01 AM,"136 Church St, New York City, NY 10001",New York City,Laptop


In [49]:
query_3_1="""
CREATE OR REPLACE TABLE sales AS 
    SELECT 
        "Order ID"::INTEGER AS order_id,
        "Product" AS "product",
        "Quantity Ordered":: INTEGER AS quantity,
        "Price" :: DECIMAL AS price_each,
        "Order Date" AS order_date, 
        "Purchase Address" AS purchase_address
    FROM df_2
    WHERE 
        TRY_CAST ("Order ID" AS integer) NOTNULL
        AND TRY_CAST("Price" AS DECIMAL(10,2)) NOT NULL
"""

conn.execute(query_3_1).df()

,Count
0,181226


In [50]:
df_sales=conn.execute("FROM sales").df()

In [51]:
df_sales.sample(5)

,order_id,product,quantity,price_each,order_date,purchase_address
24622,243348,AAA Batteries (4-pack),1,2.99,2019-08-10,"160 Willow St, Seattle, WA 98101"
129468,206447,Wired Headphones,1,11.99,2019-05-25,"406 Spruce St, Atlanta, GA 30301"
93126,214045,USB-C Charging Cable,1,11.95,2019-06-19,"493 6th St, Seattle, WA 98101"
48824,314713,Wired Headphones,1,11.99,2019-12-08,"377 7th St, New York City, NY 10001"
31141,297274,Flatscreen TV,1,300.00,2019-12-25,"737 Madison St, Boston, MA 02215"


In [52]:
df_sales.price_each.min()

2.99

In [53]:
#En esta query queremos el producto con valor minimo en todas las columnas de la tabla "Sales" excluyendo algunas 
query_4="""
SELECT 
    MIN(COLUMNS(* EXCLUDE (product, order_date, purchase_address)))
FROM sales
"""
conn.execute(query_4).df()

,order_id,quantity,price_each
0,141234,1,2.99


In [60]:
# Creamos una view donde solo tengamos aquellas ordenes con 1 producto 
query_5= """
CREATE OR REPLACE VIEW agregated_sales AS
SELECT 
    order_id,
    COUNT(1) as nb_orders,
    MONTH (order_date) as month,
    str_split ( purchase_address,',')[2] AS city,
    SUM(quantity*price_each) AS reveneu
FROM sales
GROUP BY ALL 
"""
conn.execute(query_5).df()
conn.execute("FROM agregated_sales").df()

,order_id,nb_orders,month,city,reveneu
0,199979,1,5,New York City,11.99
1,199980,1,5,Dallas,11.95
2,199987,1,5,Atlanta,14.95
3,199996,1,5,San Francisco,600.00
4,200001,1,5,Atlanta,379.99
...,...,...,...,...,...
173901,199927,1,5,Los Angeles,11.95
173902,199935,1,5,Atlanta,14.95
173903,199938,1,5,New York City,150.00
173904,199940,1,5,Los Angeles,14.95


In [69]:
#  seleeccionamo aquellas ciudades con mayor ventas ordenados de manera descendente.
query_6=""" 
SELECT
    city,
    SUM(reveneu) as total
FROM  agregated_sales
GROUP BY city
ORDER BY total DESC
"""
conn.execute(query_6).df()

,city,total
0,San Francisco,6331003.91
1,Los Angeles,4175870.80
2,New York City,3547417.43
3,Boston,2847342.01
4,Atlanta,2151198.58
5,Seattle,2142555.48
6,Dallas,2118575.40
7,Portland,1747590.61
8,Austin,1392881.75


In [74]:
# creamos un archivo .parquet que nos permite almacenamiento columnar de los datos:
query_7= """
COPY (FROM agregated_sales) TO 'agregated_sales.parquet' (FORMAT PARQUET)
"""
conn.execute(query_7)